In [ ]:
!pip install googletrans==4.0.0-rc1

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 2.9 MB/s eta 0:00:00
  Created wheel for googletrans: filename=googletrans-4.0.0rc1-py3-none-any.whl size=17395 sha256=d6b371436f27c66ce3482afd5d7ca0fc3b158bbc6d9fcee3a43e85739c03a871
  Stored in directory: /root/.cache/pip/wheels/c0/59/9f/7372f0cf70160fe61b528532e1a7c8498c4becd6bcffb022de
Successfully built googletrans
  Attempting uninstall: chardet
    Found existing installation: chardet 5.2.0
    Uninstalling ch

In [ ]:
!pip install regex


In [ ]:
import requests
from bs4 import BeautifulSoup
import re
import nltk
import pandas as pd

nltk.download('punkt')

def clean_text(text):
    # Remove any alphabetic characters and keep only Hindi characters, spaces, and '|'
    cleaned_text = re.sub(r'[a-zA-Z0-9]+', '', text)
    cleaned_text = re.sub(r'[^\u0900-\u097F\s।]', '', cleaned_text)
    return cleaned_text

def extract_text(url):
    # Fetch the content from the URL
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    # Find all paragraphs
    paragraphs = soup.find_all('p')

    # Clean the introductory paragraph
    intro_paragraph = clean_text(paragraphs[0].get_text())

    # Extract and clean text from the remaining paragraphs
    extracted_text = [clean_text(p.get_text()) for p in paragraphs[1:] if re.search(r'[a-zA-Z]', p.get_text()) is None]
    return intro_paragraph, extracted_text, soup.title.string

# List of URLs to process
urls = [



]

# DataFrame to store results
data = {
    "Sentence ID": [],
    "Sentence": [],
    "Title Name": [],
    "URL": []
}

# Process each URL
sentence_id = 5600
for url in urls:
    intro_paragraph, extracted_text, title_name = extract_text(url)

    # Split the extracted text into sentences where '|' occurs
    sentences = []
    for text in extracted_text:
        sentences.extend([sentence.strip() for sentence in text.split('।') if sentence.strip()])

    # Print URL and introduction paragraph (optional)
    print("URL:", url)
    print("Introduction:")
    print(intro_paragraph)
    print("\nSentences:")

    # Store sentences and generate sentence IDs
    for sentence in sentences:
        print(sentence)  # Print each sentence (optional)
        data["Sentence ID"].append(sentence_id)
        data["Sentence"].append(sentence)
        data["Title Name"].append(title_name)
        data["URL"].append(url)
        sentence_id += 1
    print("\n\n")

# Create a DataFrame from the collected data
df = pd.DataFrame(data)

df_filtered = df[df['Sentence'].apply(lambda x: len(x.split()) >= 4)]

# Print the filtered DataFrame
print(df_filtered)

# Save the DataFrame to an Excel file
df_filtered.to_excel('extracted_sentences_with_title_filtered.xlsx', index=False)

print("Data has been saved to extracted_sentences_with_title_filtered.xlsx")


In [ ]:
import requests
from bs4 import BeautifulSoup
import re
import nltk
import pandas as pd
from googletrans import Translator

nltk.download('punkt')

def clean_text(text):
    # Remove any alphabetic characters and keep only Hindi characters, spaces, and '|'
    cleaned_text = re.sub(r'[a-zA-Z0-9]+', '', text)
    cleaned_text = re.sub(r'[^\u0900-\u097F\s।]', '', cleaned_text)
    return cleaned_text

def extract_text(url):
    # Fetch the content from the URL
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    # Find all paragraphs
    paragraphs = soup.find_all('p')

    # Clean the introductory paragraph
    intro_paragraph = clean_text(paragraphs[0].get_text())

    # Extract and clean text from the remaining paragraphs
    extracted_text = [clean_text(p.get_text()) for p in paragraphs[1:] if re.search(r'[a-zA-Z]', p.get_text()) is None]

    # Extract Hindi title from the <title> tag
    hindi_title = soup.title.string

    # Translate Hindi title to English using googletrans
    translator = Translator()
    english_title = translator.translate(hindi_title, src='hi', dest='en').text

    # Remove "Wikipedia" from the English title
    english_title = english_title.split(" - ")[0]

    # Extract only the Hindi part of the title
    hindi_title = hindi_title.split(" - ")[0]

    return intro_paragraph, extracted_text, hindi_title, english_title

# List of URLs to process
urls = [

    'https://hi.wikipedia.org/wiki/%E0%A4%95%E0%A4%AE%E0%A5%8D%E0%A4%AA%E0%A5%8D%E0%A4%AF%E0%A5%82%E0%A4%9F%E0%A4%B0_%E0%A4%B5%E0%A4%BF%E0%A4%9C%E0%A5%8D%E0%A4%9E%E0%A4%BE%E0%A4%A8',
    'https://hi.wikipedia.org/wiki/%E0%A4%9C%E0%A4%B2%E0%A4%B5%E0%A4%BF%E0%A4%9C%E0%A5%8D%E0%A4%9E%E0%A4%BE%E0%A4%A8',
    'https://hi.wikipedia.org/wiki/%E0%A4%96%E0%A4%A8%E0%A4%BF%E0%A4%9C_%E0%A4%B5%E0%A4%BF%E0%A4%9C%E0%A5%8D%E0%A4%9E%E0%A4%BE%E0%A4%A8',
    'https://hi.wikipedia.org/wiki/%E0%A4%AD%E0%A5%82%E0%A4%B5%E0%A4%BF%E0%A4%9C%E0%A5%8D%E0%A4%9E%E0%A4%BE%E0%A4%A8',
    'https://hi.wikipedia.org/wiki/%E0%A4%96%E0%A4%97%E0%A5%8B%E0%A4%B2_%E0%A4%B6%E0%A4%BE%E0%A4%B8%E0%A5%8D%E0%A4%A4%E0%A5%8D%E0%A4%B0',
    'https://hi.wikipedia.org/wiki/%E0%A4%A8%E0%A4%BE%E0%A4%AD%E0%A4%BF%E0%A4%95%E0%A5%80%E0%A4%AF_%E0%A4%B6%E0%A4%95%E0%A5%8D%E0%A4%A4%E0%A4%BF',
    'https://hi.wikipedia.org/wiki/%E0%A4%9C%E0%A5%88%E0%A4%B5%E0%A4%AA%E0%A5%8D%E0%A4%B0%E0%A5%8C%E0%A4%A6%E0%A5%8D%E0%A4%AF%E0%A5%8B%E0%A4%97%E0%A4%BF%E0%A4%95%E0%A5%80',
    'https://hi.wikipedia.org/wiki/%E0%A4%9A%E0%A4%BF%E0%A4%95%E0%A4%BF%E0%A4%A4%E0%A5%8D%E0%A4%B8%E0%A4%BE',
    'https://hi.wikipedia.org/wiki/%E0%A4%AA%E0%A4%B0%E0%A5%8D%E0%A4%AF%E0%A4%BE%E0%A4%B5%E0%A4%B0%E0%A4%A3%E0%A4%B5%E0%A4%BF%E0%A4%9C%E0%A5%8D%E0%A4%9E%E0%A4%BE%E0%A4%A8',
    'https://hi.wikipedia.org/wiki/%E0%A4%95%E0%A4%82%E0%A4%AA%E0%A5%8D%E0%A4%AF%E0%A5%82%E0%A4%9F%E0%A4%B0',



    'https://hi.wikipedia.org/wiki/%E0%A4%85%E0%A4%82%E0%A4%A4%E0%A4%B0%E0%A4%BF%E0%A4%95%E0%A5%8D%E0%A4%B7_%E0%A4%B5%E0%A4%BF%E0%A4%9C%E0%A5%8D%E0%A4%9E%E0%A4%BE%E0%A4%A8',
    'https://hi.wikipedia.org/wiki/%E0%A4%97%E0%A4%A3%E0%A4%BF%E0%A4%A4',
    'https://hi.wikipedia.org/wiki/%E0%A4%B0%E0%A4%B8%E0%A4%BE%E0%A4%AF%E0%A4%A8_%E0%A4%B5%E0%A4%BF%E0%A4%9C%E0%A5%8D%E0%A4%9E%E0%A4%BE%E0%A4%A8',








    'https://hi.wikipedia.org/wiki/%E0%A4%9C%E0%A5%88%E0%A4%B5_%E0%A4%B8%E0%A5%82%E0%A4%9A%E0%A4%A8%E0%A4%BE_%E0%A4%B5%E0%A4%BF%E0%A4%9C%E0%A5%8D%E0%A4%9E%E0%A4%BE%E0%A4%A8',
    'https://hi.wikipedia.org/wiki/%E0%A4%AE%E0%A5%8B%E0%A4%9F%E0%A4%B0%E0%A4%B5%E0%A4%BE%E0%A4%B9%E0%A4%A8',
    'https://hi.wikipedia.org/wiki/%E0%A4%B8%E0%A4%82%E0%A4%A7%E0%A4%BE%E0%A4%B0%E0%A4%A3%E0%A5%80%E0%A4%AF_%E0%A4%8A%E0%A4%B0%E0%A5%8D%E0%A4%9C%E0%A4%BE',
    'https://hi.wikipedia.org/wiki/%E0%A4%B8%E0%A5%8D%E0%A4%B5%E0%A4%BE%E0%A4%B8%E0%A5%8D%E0%A4%A5%E0%A5%8D%E0%A4%AF_%E0%A4%B8%E0%A5%87%E0%A4%B5%E0%A4%BE_%E0%A4%AE%E0%A5%87%E0%A4%82_%E0%A4%95%E0%A5%83%E0%A4%A4%E0%A5%8D%E0%A4%B0%E0%A4%BF%E0%A4%AE_%E0%A4%AC%E0%A5%81%E0%A4%A6%E0%A5%8D%E0%A4%A7%E0%A4%BF%E0%A4%AE%E0%A4%A4%E0%A5%8D%E0%A4%A4%E0%A4%BE',
    'https://hi.wikipedia.org/wiki/%E0%A4%B8%E0%A5%8D%E0%A4%A5%E0%A4%BE%E0%A4%88_%E0%A4%95%E0%A5%83%E0%A4%B7%E0%A4%BF',
    'https://hi.wikipedia.org/wiki/%E0%A4%AD%E0%A4%BE%E0%A4%B0%E0%A4%A4_%E0%A4%AE%E0%A5%87%E0%A4%82_%E0%A4%B8%E0%A5%8D%E0%A4%AE%E0%A4%BE%E0%A4%B0%E0%A5%8D%E0%A4%9F_%E0%A4%A8%E0%A4%97%E0%A4%B0',
    'https://hi.wikipedia.org/wiki/%E0%A4%85%E0%A4%A8%E0%A5%81%E0%A4%B5%E0%A4%82%E0%A4%B6%E0%A4%BF%E0%A4%95_%E0%A4%85%E0%A4%AD%E0%A4%BF%E0%A4%AF%E0%A4%BE%E0%A4%82%E0%A4%A4%E0%A5%8D%E0%A4%B0%E0%A4%BF%E0%A4%95%E0%A5%80#:~:text=%E0%A4%9C%E0%A5%88%E0%A4%B5%20%E0%A4%AA%E0%A5%8D%E0%A4%B0%E0%A5%8C%E0%A4%A6%E0%A5%8D%E0%A4%AF%E0%A5%8B%E0%A4%97%E0%A4%BF%E0%A4%95%E0%A5%80%20%E0%A4%A6%E0%A5%8D%E0%A4%B5%E0%A4%BE%E0%A4%B0%E0%A4%BE%20%E0%A4%95%E0%A4%BF%E0%A4%B8%E0%A5%80%20%E0%A4%AD%E0%A5%80,%E0%A4%AA%E0%A5%8D%E0%A4%B0%E0%A5%8C%E0%A4%A6%E0%A5%8D%E0%A4%AF%E0%A5%8B%E0%A4%97%E0%A4%BF%E0%A4%95%E0%A4%BF%E0%A4%AF%E0%A5%8B%E0%A4%82%20%E0%A4%95%E0%A4%BE%20%E0%A4%8F%E0%A4%95%20%E0%A4%B8%E0%A5%87%E0%A4%9F%20%E0%A4%B9%E0%A5%88%E0%A5%A4',
    'https://hi.wikipedia.org/wiki/%E0%A4%B5%E0%A4%B8%E0%A5%8D%E0%A4%A4%E0%A5%81_%E0%A4%85%E0%A4%82%E0%A4%A4%E0%A4%B0%E0%A4%9C%E0%A4%BE%E0%A4%B2#:~:text=%E0%A4%87%E0%A4%82%E0%A4%9F%E0%A4%B0%E0%A4%A8%E0%A5%87%E0%A4%9F%20%E0%A4%91%E0%A4%AB%E0%A4%BC%20%E0%A4%A5%E0%A4%BF%E0%A4%82%E0%A4%97%E0%A5%8D%E0%A4%B8%20(Internet%20of,%E0%A4%9C%E0%A5%81%E0%A4%A1%E0%A4%BC%E0%A4%A8%E0%A5%87%20%E0%A4%95%E0%A5%80%20%E0%A4%95%E0%A5%8D%E0%A4%B7%E0%A4%AE%E0%A4%A4%E0%A4%BE%20%E0%A4%B9%E0%A5%8B%E0%A4%A4%E0%A5%80%20%E0%A4%B9%E0%A5%88%E0%A5%A4',
    'https://hi.wikipedia.org/wiki/%E0%A4%9C%E0%A5%88%E0%A4%B5_%E0%A4%B5%E0%A4%BF%E0%A4%B5%E0%A4%BF%E0%A4%A7%E0%A4%A4%E0%A4%BE',
    'https://hi.wikipedia.org/wiki/%E0%A4%B8%E0%A5%81%E0%A4%A6%E0%A5%82%E0%A4%B0_%E0%A4%B8%E0%A4%82%E0%A4%B5%E0%A5%87%E0%A4%A6%E0%A4%A8',
    'https://hi.wikipedia.org/wiki/%E0%A4%9F%E0%A5%8D%E0%A4%B0%E0%A5%88%E0%A4%AB%E0%A4%BF%E0%A4%95_%E0%A4%B2%E0%A4%BE%E0%A4%87%E0%A4%9F',
    'https://hi.wikipedia.org/wiki/%E0%A4%AE%E0%A5%8C%E0%A4%B8%E0%A4%AE_%E0%A4%B5%E0%A4%BF%E0%A4%9C%E0%A5%8D%E0%A4%9E%E0%A4%BE%E0%A4%A8',
    'https://hi.wikipedia.org/wiki/%E0%A4%AE%E0%A5%8C%E0%A4%B8%E0%A4%AE',
    'https://hi.wikipedia.org/wiki/%E0%A4%B8%E0%A5%89%E0%A4%AB%E0%A5%8D%E0%A4%9F%E0%A4%B5%E0%A5%87%E0%A4%AF%E0%A4%B0_%E0%A4%85%E0%A4%AD%E0%A4%BF%E0%A4%AF%E0%A4%BE%E0%A4%A8%E0%A5%8D%E0%A4%A4%E0%A5%8D%E0%A4%B0%E0%A4%BF%E0%A4%95%E0%A5%80#:~:text=%E0%A4%B8%E0%A5%89%E0%A4%AB%E0%A5%8D%E0%A4%9F%E0%A4%B5%E0%A5%87%E0%A4%AF%E0%A4%B0%20%E0%A4%85%E0%A4%AD%E0%A4%BF%E0%A4%AF%E0%A4%BE%E0%A4%A8%E0%A5%8D%E0%A4%A4%E0%A5%8D%E0%A4%B0%E0%A4%BF%E0%A4%95%E0%A5%80%20(Software%20Engineering)%20%E0%A4%85%E0%A4%AD%E0%A4%BF%E0%A4%AF%E0%A4%BE%E0%A4%A8%E0%A5%8D%E0%A4%A4%E0%A5%8D%E0%A4%B0%E0%A4%BF%E0%A4%95%E0%A5%80,%E0%A4%95%E0%A4%BE%20%E0%A4%85%E0%A4%A7%E0%A5%8D%E0%A4%AF%E0%A4%AF%E0%A4%A8%20%E0%A4%95%E0%A4%BF%E0%A4%AF%E0%A4%BE%20%E0%A4%9C%E0%A4%BE%E0%A4%A4%E0%A4%BE%20%E0%A4%B9%E0%A5%88%E0%A5%A4',
    'https://hi.wikipedia.org/wiki/%E0%A4%93%E0%A4%B8%E0%A5%80%E0%A4%86%E0%A4%B0',
    'https://hi.wikipedia.org/wiki/%E0%A4%9C%E0%A4%B2%E0%A4%B5%E0%A4%BE%E0%A4%AF%E0%A5%81%E0%A4%B5%E0%A4%BF%E0%A4%9C%E0%A5%8D%E0%A4%9E%E0%A4%BE%E0%A4%A8#:~:text=%E0%A4%9C%E0%A4%B2%E0%A4%B5%E0%A4%BE%E0%A4%AF%E0%A5%81%20%E0%A4%B5%E0%A4%BF%E0%A4%9C%E0%A5%8D%E0%A4%9E%E0%A4%BE%E0%A4%A8%20(climatology)%20%E0%A4%AD%E0%A5%8C%E0%A4%A4%E0%A4%BF%E0%A4%95%20%E0%A4%AD%E0%A5%82%E0%A4%97%E0%A5%8B%E0%A4%B2,%E0%A4%B5%E0%A4%BF%E0%A4%AD%E0%A4%BF%E0%A4%A8%E0%A4%A4%E0%A4%BE%20%E0%A4%95%E0%A5%80%20%E0%A4%95%E0%A5%81%E0%A4%82%E0%A4%9C%E0%A5%80%20%E0%A4%95%E0%A4%B9%E0%A4%A4%E0%A5%87%20%E0%A4%B9%E0%A5%88%E0%A4%82%E0%A5%A4',
    'https://hi.wikipedia.org/wiki/%E0%A4%85%E0%A4%82%E0%A4%A4%E0%A4%B0%E0%A4%BF%E0%A4%95%E0%A5%8D%E0%A4%B7',
    'https://hi.wikipedia.org/wiki/%E0%A4%85%E0%A4%A8%E0%A5%8D%E0%A4%A4%E0%A4%B0%E0%A4%BF%E0%A4%95%E0%A5%8D%E0%A4%B7',
    'https://hi.wikipedia.org/wiki/%E0%A4%85%E0%A4%A8%E0%A5%8D%E0%A4%A4%E0%A4%B0%E0%A4%BF%E0%A4%95%E0%A5%8D%E0%A4%B7%E0%A5%80%E0%A4%AF_%E0%A4%85%E0%A4%AD%E0%A4%BF%E0%A4%AF%E0%A4%BE%E0%A4%A8%E0%A5%8D%E0%A4%A4%E0%A5%8D%E0%A4%B0%E0%A4%BF%E0%A4%95%E0%A5%80#:~:text=%E0%A4%85%E0%A4%A8%E0%A5%8D%E0%A4%A4%E0%A4%B0%E0%A4%BF%E0%A4%95%E0%A5%8D%E0%A4%B7%E0%A5%80%E0%A4%AF%20%E0%A4%85%E0%A4%AD%E0%A4%BF%E0%A4%AF%E0%A4%BE%E0%A4%A8%E0%A5%8D%E0%A4%A4%E0%A5%8D%E0%A4%B0%E0%A4%BF%E0%A4%95%E0%A5%80%20%E0%A4%B5%E0%A4%B9%20%E0%A4%B6%E0%A4%BE%E0%A4%96%E0%A4%BE%20%E0%A4%B9%E0%A5%88,%E0%A4%AE%E0%A5%87%E0%A4%82%20%E0%A4%B8%E0%A4%82%E0%A4%9A%E0%A4%BE%E0%A4%B0%20%E0%A4%B6%E0%A4%BE%E0%A4%AE%E0%A4%BF%E0%A4%B2%20%E0%A4%B9%E0%A5%8B%E0%A4%A4%E0%A4%BE%20%E0%A4%B9%E0%A5%88%E0%A5%A4',
    'https://hi.wikipedia.org/wiki/%E0%A4%86%E0%A4%A8%E0%A5%81%E0%A4%B5%E0%A4%82%E0%A4%B6%E0%A4%BF%E0%A4%95%E0%A5%80',
    'https://hi.wikipedia.org/wiki/%E0%A4%AD%E0%A5%82%E0%A4%AE%E0%A4%82%E0%A4%A1%E0%A4%B2%E0%A5%80%E0%A4%AF_%E0%A4%8A%E0%A4%B7%E0%A5%8D%E0%A4%AE%E0%A5%80%E0%A4%95%E0%A4%B0%E0%A4%A3',
    'https://hi.wikipedia.org/wiki/%E0%A4%B8%E0%A4%82%E0%A4%A7%E0%A4%BE%E0%A4%B0%E0%A4%A3%E0%A5%80%E0%A4%AF_%E0%A4%B5%E0%A4%BF%E0%A4%95%E0%A4%BE%E0%A4%B8_%E0%A4%B2%E0%A4%95%E0%A5%8D%E0%A4%B7%E0%A5%8D%E0%A4%AF',
    'https://hi.wikipedia.org/wiki/%E0%A4%AE%E0%A4%A8%E0%A5%8B%E0%A4%B5%E0%A4%BF%E0%A4%9C%E0%A5%8D%E0%A4%9E%E0%A4%BE%E0%A4%A8',
    'https://hi.wikipedia.org/wiki/%E0%A4%AA%E0%A4%BE%E0%A4%B0%E0%A4%BF%E0%A4%B8%E0%A5%8D%E0%A4%A5%E0%A4%BF%E0%A4%A4%E0%A4%BF%E0%A4%95%E0%A5%80',
    'https://hi.wikipedia.org/wiki/%E0%A4%AE%E0%A4%BE%E0%A4%A8%E0%A4%B5_%E0%A4%AA%E0%A4%BE%E0%A4%B0%E0%A4%BF%E0%A4%B8%E0%A5%8D%E0%A4%A5%E0%A4%BF%E0%A4%A4%E0%A4%BF%E0%A4%95%E0%A5%80',
    'https://hi.wikipedia.org/wiki/%E0%A4%B8%E0%A4%BE%E0%A4%AE%E0%A4%BE%E0%A4%9C%E0%A4%BF%E0%A4%95_%E0%A4%AE%E0%A5%80%E0%A4%A1%E0%A4%BF%E0%A4%AF%E0%A4%BE',
    'https://bh.wikipedia.org/wiki/%E0%A4%B8%E0%A5%8B%E0%A4%B6%E0%A4%B2_%E0%A4%AE%E0%A5%80%E0%A4%A1%E0%A4%BF%E0%A4%AF%E0%A4%BE',
    'https://bh.wikipedia.org/wiki/%E0%A4%9C%E0%A4%B2%E0%A4%B5%E0%A4%BE%E0%A4%AF%E0%A5%81_%E0%A4%AC%E0%A4%A6%E0%A4%B2%E0%A4%BE%E0%A4%B5',
    'https://hi.wikipedia.org/wiki/%E0%A4%B8%E0%A4%AE%E0%A4%BE%E0%A4%9C_%E0%A4%AE%E0%A4%A8%E0%A5%8B%E0%A4%B5%E0%A4%BF%E0%A4%9C%E0%A5%8D%E0%A4%9E%E0%A4%BE%E0%A4%A8',
    'https://hi.wikipedia.org/wiki/%E0%A4%95%E0%A5%8D%E0%A4%B0%E0%A4%AE-%E0%A4%B5%E0%A4%BF%E0%A4%95%E0%A4%BE%E0%A4%B8',
    'https://hi.wikipedia.org/wiki/%E0%A4%95%E0%A5%8D%E0%A4%B0%E0%A4%AE-%E0%A4%B5%E0%A4%BF%E0%A4%95%E0%A4%BE%E0%A4%B8_%E0%A4%B8%E0%A5%87_%E0%A4%AA%E0%A4%B0%E0%A4%BF%E0%A4%9A%E0%A4%AF',





    'https://hi.wikipedia.org/wiki/%E0%A4%B0%E0%A5%87%E0%A4%A1%E0%A4%BF%E0%A4%AF%E0%A5%8B',
    'https://hi.wikipedia.org/wiki/%E0%A4%86%E0%A4%AF%E0%A5%81%E0%A4%B0%E0%A5%8D%E0%A4%B5%E0%A4%BF%E0%A4%9C%E0%A5%8D%E0%A4%9E%E0%A4%BE%E0%A4%A8',

    'https://hi.wikipedia.org/wiki/%E0%A4%B9%E0%A5%83%E0%A4%A6%E0%A4%AF',
    'https://hi.wikipedia.org/wiki/%E0%A4%B8%E0%A4%BE%E0%A4%82%E0%A4%96%E0%A5%8D%E0%A4%AF%E0%A4%BF%E0%A4%95%E0%A5%80',

    'https://hi.wikipedia.org/wiki/%E0%A4%AE%E0%A4%BE%E0%A4%A8%E0%A4%B5%E0%A4%B6%E0%A4%BE%E0%A4%B8%E0%A5%8D%E0%A4%A4%E0%A5%8D%E0%A4%B0',
    'https://hi.wikipedia.org/wiki/%E0%A4%B8%E0%A4%AE%E0%A4%BE%E0%A4%9C%E0%A4%B6%E0%A4%BE%E0%A4%B8%E0%A5%8D%E0%A4%A4%E0%A5%8D%E0%A4%B0',
    'https://hi.wikipedia.org/wiki/%E0%A4%B8%E0%A5%8D%E0%A4%B5%E0%A4%BE%E0%A4%B8%E0%A5%8D%E0%A4%A5%E0%A5%8D%E0%A4%AF_%E0%A4%B5%E0%A4%BF%E0%A4%9C%E0%A5%8D%E0%A4%9E%E0%A4%BE%E0%A4%A8',
    'https://hi.wikipedia.org/wiki/%E0%A4%AD%E0%A5%8C%E0%A4%A4%E0%A4%BF%E0%A4%95_%E0%A4%B6%E0%A4%BE%E0%A4%B8%E0%A5%8D%E0%A4%A4%E0%A5%8D%E0%A4%B0',
    'https://hi.wikipedia.org/wiki/%E0%A4%85%E0%A4%B0%E0%A5%8D%E0%A4%A5%E0%A4%B6%E0%A4%BE%E0%A4%B8%E0%A5%8D%E0%A4%A4%E0%A5%8D%E0%A4%B0',
    'https://hi.wikipedia.org/wiki/%E0%A4%B6%E0%A4%BF%E0%A4%95%E0%A5%8D%E0%A4%B7%E0%A4%BE%E0%A4%B6%E0%A4%BE%E0%A4%B8%E0%A5%8D%E0%A4%A4%E0%A5%8D%E0%A4%B0',
    'https://hi.wikipedia.org/wiki/%E0%A4%AD%E0%A5%82%E0%A4%97%E0%A5%8B%E0%A4%B2',
    'https://hi.wikipedia.org/wiki/%E0%A4%87%E0%A4%A4%E0%A4%BF%E0%A4%B9%E0%A4%BE%E0%A4%B8',
    'https://hi.wikipedia.org/wiki/%E0%A4%AD%E0%A4%BE%E0%A4%B7%E0%A4%BE%E0%A4%B5%E0%A4%BF%E0%A4%9C%E0%A5%8D%E0%A4%9E%E0%A4%BE%E0%A4%A8',
    'https://hi.wikipedia.org/wiki/%E0%A4%9C%E0%A4%B2%E0%A4%B5%E0%A4%BE%E0%A4%AF%E0%A5%81_%E0%A4%AA%E0%A4%B0%E0%A4%BF%E0%A4%B5%E0%A4%B0%E0%A5%8D%E0%A4%A4%E0%A4%A8',
    'https://hi.wikipedia.org/wiki/%E0%A4%B8%E0%A4%BE%E0%A4%AE%E0%A4%BE%E0%A4%9C%E0%A4%BF%E0%A4%95_%E0%A4%A8%E0%A5%8D%E0%A4%AF%E0%A4%BE%E0%A4%AF',
    'https://hi.wikipedia.org/wiki/%E0%A4%AE%E0%A4%B8%E0%A5%8D%E0%A4%A4%E0%A4%BF%E0%A4%B7%E0%A5%8D%E0%A4%95',
    'https://hi.wikipedia.org/wiki/%E0%A4%B8%E0%A5%8C%E0%A4%B0_%E0%A4%8A%E0%A4%B0%E0%A5%8D%E0%A4%9C%E0%A4%BE',
    'https://hi.wikipedia.org/wiki/%E0%A4%B8%E0%A5%8C%E0%A4%B0_%E0%A4%AE%E0%A4%A3%E0%A5%8D%E0%A4%A1%E0%A4%B2',
    'https://hi.wikipedia.org/wiki/%E0%A4%97%E0%A5%8D%E0%A4%B0%E0%A4%B9',
    'https://hi.wikipedia.org/wiki/%E0%A4%9A%E0%A4%A8%E0%A5%8D%E0%A4%A6%E0%A5%8D%E0%A4%B0%E0%A4%AE%E0%A4%BE',
    'https://hi.wikipedia.org/wiki/%E0%A4%AE%E0%A4%82%E0%A4%97%E0%A4%B2_%E0%A4%97%E0%A5%8D%E0%A4%B0%E0%A4%B9',
    'https://hi.wikipedia.org/wiki/%E0%A4%AC%E0%A5%81%E0%A4%A7_(%E0%A4%97%E0%A5%8D%E0%A4%B0%E0%A4%B9)',
    'https://hi.wikipedia.org/wiki/%E0%A4%97%E0%A5%81%E0%A4%B0%E0%A5%81%E0%A4%A4%E0%A5%8D%E0%A4%B5%E0%A4%BE%E0%A4%95%E0%A4%B0%E0%A5%8D%E0%A4%B7%E0%A4%A3',
    'https://hi.wikipedia.org/wiki/%E0%A4%89%E0%A4%AA%E0%A4%97%E0%A5%8D%E0%A4%B0%E0%A4%B9',
    'https://hi.wikipedia.org/wiki/%E0%A4%A8%E0%A4%95%E0%A5%8D%E0%A4%B7%E0%A4%A4%E0%A5%8D%E0%A4%B0',
    'https://hi.wikipedia.org/wiki/%E0%A4%86%E0%A4%95%E0%A4%BE%E0%A4%B6%E0%A4%97%E0%A4%82%E0%A4%97%E0%A4%BE',
    'https://hi.wikipedia.org/wiki/%E0%A4%A4%E0%A4%BE%E0%A4%B0%E0%A4%BE',
    'https://hi.wikipedia.org/wiki/%E0%A4%B8%E0%A5%82%E0%A4%B0%E0%A5%8D%E0%A4%AF',
    'https://hi.wikipedia.org/wiki/%E0%A4%95%E0%A4%BE%E0%A4%B2%E0%A5%8D%E0%A4%AA%E0%A4%A8%E0%A4%BF%E0%A4%95_%E0%A4%AF%E0%A4%BE%E0%A4%A4%E0%A5%8D%E0%A4%B0%E0%A4%BE',







    'https://hi.wikipedia.org/wiki/%E0%A4%B6%E0%A4%B2%E0%A5%8D%E0%A4%AF%E0%A4%9A%E0%A4%BF%E0%A4%95%E0%A4%BF%E0%A4%A4%E0%A5%8D%E0%A4%B8%E0%A4%BE',


    'https://hi.wikipedia.org/wiki/%E0%A4%9C%E0%A5%80%E0%A4%B5_%E0%A4%B5%E0%A4%BF%E0%A4%9C%E0%A5%8D%E0%A4%9E%E0%A4%BE%E0%A4%A8',
    'https://hi.wikipedia.org/wiki/%E0%A4%A6%E0%A4%A8%E0%A5%8D%E0%A4%A4%E0%A4%9A%E0%A4%BF%E0%A4%95%E0%A4%BF%E0%A4%A4%E0%A5%8D%E0%A4%B8%E0%A4%BE',


    'https://hi.wikipedia.org/wiki/%E0%A4%AA%E0%A5%8D%E0%A4%B0%E0%A4%BE%E0%A4%95%E0%A5%83%E0%A4%A4%E0%A4%BF%E0%A4%95_%E0%A4%AD%E0%A4%BE%E0%A4%B7%E0%A4%BE_%E0%A4%B8%E0%A4%82%E0%A4%B8%E0%A4%BE%E0%A4%A7%E0%A4%A8',
    'https://hi.wikipedia.org/wiki/%E0%A4%AA%E0%A5%8D%E0%A4%B0%E0%A5%8B%E0%A4%97%E0%A5%8D%E0%A4%B0%E0%A4%BE%E0%A4%AE%E0%A4%BF%E0%A4%82%E0%A4%97_%E0%A4%AD%E0%A4%BE%E0%A4%B7%E0%A4%BE',
    'https://hi.wikipedia.org/wiki/%E0%A4%95%E0%A5%83%E0%A4%A4%E0%A5%8D%E0%A4%B0%E0%A4%BF%E0%A4%AE_%E0%A4%AC%E0%A5%81%E0%A4%A6%E0%A5%8D%E0%A4%A7%E0%A4%BF',
    'https://hi.wikipedia.org/wiki/%E0%A4%87%E0%A4%B2%E0%A5%88%E0%A4%95%E0%A5%8D%E0%A4%9F%E0%A5%8D%E0%A4%B0%E0%A5%89%E0%A4%A8%E0%A4%BF%E0%A4%95%E0%A5%8D%E0%A4%B8',
    'https://hi.wikipedia.org/wiki/%E0%A4%B0%E0%A5%8B%E0%A4%AC%E0%A5%89%E0%A4%9F%E0%A4%BF%E0%A4%95%E0%A5%8D%E0%A4%B8',
    'https://hi.wikipedia.org/wiki/%E0%A4%B5%E0%A4%BF%E0%A4%A6%E0%A5%8D%E0%A4%AF%E0%A5%81%E0%A4%A4_%E0%A4%A7%E0%A4%BE%E0%A4%B0%E0%A4%BE',
    'https://hi.wikipedia.org/wiki/%E0%A4%B8%E0%A5%87%E0%A4%82%E0%A4%B8%E0%A4%B0',
    'https://hi.wikipedia.org/wiki/%E0%A4%9A%E0%A5%81%E0%A4%AE%E0%A5%8D%E0%A4%AC%E0%A4%95',
    'https://hi.wikipedia.org/wiki/%E0%A4%B8%E0%A4%AE%E0%A5%81%E0%A4%A6%E0%A5%8D%E0%A4%B0%E0%A5%80_%E0%A4%9C%E0%A5%80%E0%A4%B5%E0%A4%B5%E0%A4%BF%E0%A4%9C%E0%A5%8D%E0%A4%9E%E0%A4%BE%E0%A4%A8',
    'https://hi.wikipedia.org/wiki/%E0%A4%B8%E0%A4%AE%E0%A5%81%E0%A4%A6%E0%A5%8D%E0%A4%B0%E0%A5%80_%E0%A4%AA%E0%A5%8D%E0%A4%B0%E0%A4%A6%E0%A5%82%E0%A4%B7%E0%A4%A3',
    'https://hi.wikipedia.org/wiki/%E0%A4%B5%E0%A4%BE%E0%A4%AF%E0%A5%81%E0%A4%AF%E0%A4%BE%E0%A4%A8',
    'https://hi.wikipedia.org/wiki/%E0%A4%AE%E0%A5%8B%E0%A4%9F%E0%A4%B0%E0%A4%B8%E0%A4%BE%E0%A4%87%E0%A4%95%E0%A4%BF%E0%A4%B2',
    'https://hi.wikipedia.org/wiki/%E0%A4%B8%E0%A4%BE%E0%A4%AF%E0%A4%95%E0%A4%BF%E0%A4%B2',
    'https://hi.wikipedia.org/wiki/%E0%A4%B9%E0%A5%88%E0%A4%B2%E0%A5%80%E0%A4%95%E0%A5%89%E0%A4%AA%E0%A5%8D%E0%A4%9F%E0%A4%B0',
    'https://hi.wikipedia.org/wiki/%E0%A4%9C%E0%A4%B2%E0%A4%AF%E0%A4%BE%E0%A4%A8',
    'https://hi.wikipedia.org/wiki/%E0%A4%AC%E0%A4%B8',
    'https://hi.wikipedia.org/wiki/%E0%A4%85%E0%A4%B5%E0%A4%B0%E0%A4%95%E0%A5%8D%E0%A4%A4',
    'https://hi.wikipedia.org/wiki/%E0%A4%B8%E0%A4%BF%E0%A4%B2%E0%A4%BF%E0%A4%95%E0%A5%89%E0%A4%A8',
    'https://hi.wikipedia.org/wiki/%E0%A4%8F%E0%A4%AA%E0%A5%8D%E0%A4%AA%E0%A4%B2_%E0%A4%87%E0%A4%82%E0%A4%95%E0%A5%B0',
    'https://hi.wikipedia.org/wiki/%E0%A4%AE%E0%A4%BE%E0%A4%87%E0%A4%95%E0%A5%8D%E0%A4%B0%E0%A5%8B%E0%A4%B8%E0%A5%89%E0%A4%AB%E0%A4%BC%E0%A5%8D%E0%A4%9F',
    'https://hi.wikipedia.org/wiki/%E0%A4%97%E0%A5%82%E0%A4%97%E0%A4%B2'


    'https://hi.wikipedia.org/wiki/%E0%A4%AE%E0%A4%BE%E0%A4%A8%E0%A4%B5_%E0%A4%95%E0%A4%BE_%E0%A4%AA%E0%A5%8B%E0%A4%B7%E0%A4%A3_%E0%A4%A8%E0%A4%BE%E0%A4%B2',
    'https://hi.wikipedia.org/wiki/%E0%A4%9A%E0%A4%95%E0%A5%8D%E0%A4%B0%E0%A4%B5%E0%A4%BE%E0%A4%A4',
    'https://hi.wikipedia.org/wiki/%E0%A4%89%E0%A4%B7%E0%A5%8D%E0%A4%A3%E0%A4%95%E0%A4%9F%E0%A4%BF%E0%A4%AC%E0%A4%82%E0%A4%A7%E0%A5%80%E0%A4%AF_%E0%A4%9A%E0%A4%95%E0%A5%8D%E0%A4%B0%E0%A4%B5%E0%A4%BE%E0%A4%A4',
    'https://hi.wikipedia.org/wiki/%E0%A4%8F%E0%A4%95%E0%A5%80%E0%A4%95%E0%A5%83%E0%A4%A4_%E0%A4%AA%E0%A4%B0%E0%A4%BF%E0%A4%AA%E0%A4%A5',
    'https://hi.wikipedia.org/wiki/%E0%A4%AE%E0%A4%BE%E0%A4%87%E0%A4%95%E0%A5%8D%E0%A4%B0%E0%A5%8B%E0%A4%AA%E0%A5%8D%E0%A4%B0%E0%A5%8B%E0%A4%B8%E0%A5%87%E0%A4%B8%E0%A4%B0',



     'https://hi.wikipedia.org/wiki/%E0%A4%B5%E0%A4%BE%E0%A4%88-%E0%A4%AB%E0%A4%BC%E0%A4%BE%E0%A4%88',
    'https://hi.wikipedia.org/wiki/%E0%A4%88%E0%A4%A5%E0%A4%B0%E0%A4%A8%E0%A5%87%E0%A4%9F',
    'https://hi.wikipedia.org/wiki/%E0%A4%AD%E0%A4%BE%E0%A4%B0%E0%A4%A4_%E0%A4%95%E0%A4%BE_%E0%A4%87%E0%A4%A4%E0%A4%BF%E0%A4%B9%E0%A4%BE%E0%A4%B8',
    'https://hi.wikipedia.org/wiki/%E0%A4%AD%E0%A4%BE%E0%A4%B0%E0%A4%A4',
    'https://hi.wikipedia.org/wiki/%E0%A4%9C%E0%A5%8D%E0%A4%AF%E0%A5%8B%E0%A4%A4%E0%A4%BF%E0%A4%B7'




    # List of URLs...
]

# DataFrame to store results
data = {
    "Sentence ID": [],
    "Sentence": [],
    "Hindi Title": [],
    "English Title": [],
    "URL": []
}

# Process each URL
sentence_id = 1
for url in urls:
    intro_paragraph, extracted_text, hindi_title, english_title = extract_text(url)

    # Split the extracted text into sentences where '|' occurs
    sentences = []
    for text in extracted_text:
        sentences.extend([sentence.strip() for sentence in text.split('।') if sentence.strip()])

    # Store sentences and generate sentence IDs
    for sentence in sentences:
        data["Sentence ID"].append(sentence_id)
        data["Sentence"].append(sentence)
        data["Hindi Title"].append(hindi_title)
        data["English Title"].append(english_title)
        data["URL"].append(url)
        sentence_id += 1

# Create a DataFrame from the collected data
df = pd.DataFrame(data)

# Filter sentences with at least 4 words
df_filtered = df[df['Sentence'].apply(lambda x: len(x.split()) >= 4)]

# Print the filtered DataFrame with Hindi and English titles side by side
print(df_filtered[['Hindi Title', 'English Title', 'Sentence']])

# Save the DataFrame to an Excel file
df_filtered.to_excel('Final_sentences_with_title_filtered.xlsx', index=False)

print("Data has been saved to extracted_sentences_with_title_filtered.xlsx")


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


            Hindi Title     English Title  \
0     कम्प्यूटर विज्ञान  Computer Science   
1     कम्प्यूटर विज्ञान  Computer Science   
2     कम्प्यूटर विज्ञान  Computer Science   
3     कम्प्यूटर विज्ञान  Computer Science   
4     कम्प्यूटर विज्ञान  Computer Science   
...                 ...               ...   
6493            ज्योतिष         Astrology   
6494            ज्योतिष         Astrology   
6495            ज्योतिष         Astrology   
6496            ज्योतिष         Astrology   
6497            ज्योतिष         Astrology   

                                               Sentence  
0     कम्प्यूटर जिन भौतिक उपकरणों से मिलकर बना होता ...  
1     इनमें मॉनीटर सीपीयू कीबोर्ड माउस प्रिण्टर आदि ...  
2     हार्डवेयर से सम्बंधित विषयों का कम्प्यटूर विज्...  
3     कम्प्यूटर के प्रचालन हेतु सॉफ्टवेयर प्रोग्राम ...  
4     इनमें ऑपरेटिंग सिस्टम सिस्टम सॉफ्टवेयर तथा ऍप्...  
...                                                 ...  
6493                    राशियों का विभाग पीछे से हुआ 